* Review Application Architecture
* Performance Testing of POST requests
* Troubleshoot to identify bottleneck
* Fix the issue by adding new API
* Retest the Performance
* Exercise and Solution - POST request for orders

* Review Application Architecture

Here we will be reviewing the application architecture which is deployed in GCP VM.

1. We will be submitting the post requests from our local PC/Mac (Client).
2. The application is running in a VM provisioned from GCP. The VM is in US Central.
3. We have the Postgres Database setup using Elephant SQL. The Postgres Database is running in US Eastern.

* Performance Testing of POST requests

1. Generate Test data using Faker (100 records).
2. Make POST request for each record against the application that is running in GCP VM.

In [ ]:
from faker import Faker
faker = Faker()
users = []
usernames = set()
ctr = 0

while True:
    if ctr == 100:
        break
    first_name = faker.first_name()
    last_name = faker.last_name()
    username = f'{first_name[:1]}{last_name}'.lower()
    email = f'{username}@email.com'
    if username not in usernames:
        usernames.add(username)
        user = {
            'id': '',
            'first_name': first_name,
            'last_name': last_name,
            'username': username,
            'email': email
        }
        users.append(user)
        ctr = ctr + 1

len(users)

In [ ]:
import requests

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
for user in users:
    response = requests.post(
        f'{base_url}/user', 
        data=user
    )
    print(response.status_code)

* Troubleshoot to identify bottleneck

1. Review the overall execution time.
2. Use ping to review the response time between our PC to GCP VM and then GCP VM to the server on which Postgres is running (Elephant SQL).

* Fix the issue by adding new API

Update `user_routes/users.py` with below function:

```python
@app.route('/users', methods=['POST'])
def add_users():
    users = json.loads(request.form['users'])
    users_ = []
    for user in users:
        users_.append(User(**user))
    db.session.add_all(users_)
    db.session.commit()    
    return jsonify({'message': 'Users added successfully...'}), 201
```

* Deploy Changes to GCP VM

1. Unit test to see if `add_users` is working as expected or not.
2. Push changes to GitHub Repository.
3. Make sure the GitHub Action or CI/CD Pipeline have run with out any issues.

* Retest the Performance

1. Generate 10000 test records (without id).
2. Invoke `add_users` with 1000 records at a time.
3. Evaluate the performance.

Note: You might be able to load all the records at once. But as the number of records increase, we might exceed the size of network package and might end up running into issues. Hence, we should divide overall data into manageable chunks.

In [ ]:
from faker import Faker
faker = Faker()
users = []
usernames = set()
ctr = 0

while True:
    if ctr == 10000:
        break
    first_name = faker.first_name()
    last_name = faker.last_name()
    username = f'{first_name[:1]}{last_name}'.lower()
    email = f'{username}@email.com'
    if username not in usernames:
        usernames.add(username)
        user = {
            'first_name': first_name,
            'last_name': last_name,
            'username': username,
            'email': email
        }
        users.append(user)
        ctr = ctr + 1

len(users)

In [ ]:
import requests
import json

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
batch_size = int(input('Enter batch size: '))

In [ ]:
for i in range(0, len(users), batch_size):
    users_chunk = users[i:i+batch_size]
    response = requests.post(
        f'{base_url}/users', 
        data={'users': json.dumps(users_chunk)}
    )
    print(response.json())

In [ ]:
response.text

In [ ]:
print(response.json())

* Exercise - POST request for orders

1. Reset the tables using Flask Shell (includes orders)
2. Update `sales-app-rest` with functionality to add record into orders table. You need to develop a new function to support `POST` Methods. The function should be similar to `/user` with `POST` Method (populating one record per request).
3. Unit test for first 100 records in the file (`data/retail_db/orders`).
  * Read data CSV using Pandas `read_csv`. Make sure to specify `order_id, order_date, order_customer_id, order_status` as column names. You can drop `order_id` as it is supposed to be populated by sequence.
  * Get first 100 records by using `head` and convert them to list of dicts (using `to_dict` with `orient=records`)
  * Iterate through the list and invoke the newly added POST API.
  * Make sure to validate by either using existing end points or by invoking `Order.query.all()` using `flask shell`.
  * Make sure to push the changes to remote repository.
4. Test the performance for first 100 records in the file (`data/retail_db/orders`). You can use the same code which is used for unit testing by changing the base url to remote VM.
5. Identify the performance bottleneck.
6. Update `sales-app-rest` with functionality to add multiple records into orders table at once. Add new function similar to `/users` with POST.
7. Unit test the new function locally and push the changes to remote repository.
8. Test the performance and see if it is improved or not. Make sure to reset the tables before retesting the performance.

* Solution - POST request for orders

1. Reset the tables using Flask Shell (includes orders)

```python
db.drop_all()
db.create_all()
```

2. Update `sales-app-rest` with functionality to add record into orders table. You need to develop a new function to support `POST` Methods. Also, make sure to push changes to Git repository.

```python
@app.route('/order', methods=['POST'])
def create_or_update_order():
    """
    Create or update an order.
    ---
    parameters:
      - name: order_id
        in: formData
        type: integer
        required: false
        description: ID of the order.
      - name: order_date
        in: formData
        type: string
        format: date
        required: true
        description: Date of the order (YYYY-MM-DD).
      - name: order_customer_id
        in: formData
        type: integer
        required: true
        description: ID of the customer associated with the order.
      - name: order_status
        in: formData
        type: string
        required: true
        description: Status of the order.
    responses:
      201:
        description: Order added successfully.
      200:
        description: Order updated successfully.
    """
    order_id = request.form.get('order_id')
    order_date = request.form['order_date']
    order_customer_id = request.form['order_customer_id']
    order_status = request.form['order_status']
    if order_id:
        order = Order.query.get(order_id)
        order.order_date = order_date
        order.order_customer_id = order_customer_id
        order.order_status = order_status
        db.session.commit()
        return jsonify({'message': 'Order updated successfully...'}), 200
    else:
        order = Order(
            order_id=order_id,
            order_date=order_date,
            order_customer_id=order_customer_id,
            order_status=order_status
        )
        db.session.add(order)
        db.session.commit()
        return jsonify({'message': 'Order added successfully...'}), 201

```

3. Test the performance for first 100 records in the file (`data/retail_db/orders`).
  * Read data CSV using Pandas `read_csv`. Make sure to specify`order_id, order_date, order_customer_id, order_status` as column names.
  * Get first 100 records by using `head` and convert them to list of dicts (using `to_dict` with `orient=records`)
  * Iterate through the list and invoke the newly added POST API.
  * Make sure to validate by either using existing end points or by invoking `Order.query.all()` using `flask shell`.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(
    'data/retail_db/orders/part-00000',
    names=['order_id', 'order_date', 'order_customer_id', 'order_status']
).drop('order_id')

In [ ]:
orders = df.head(100).to_dict(orient='records')

In [ ]:
import requests

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
for order in orders:
    requests.post(f'{base_url}/order', data=order)

In [ ]:
requests.get(f'{base_url}/orders').json()

4. Identify the performance bottleneck.

Each post request which is inserting one record at a time is almost taking 1 second. We need to consider developing new function which can populate multiple records at once.

5. Update `sales-app-rest` with functionality to add multiple records into orders table at once.

Update `routes/order_routes.py` with additional function by name `add_orders` which serve `/orders` via POST method.

```python
@app.route('/orders', methods=['POST'])
def add_orders():
    orders = json.loads(request.form['orders'])
    orders_ = []
    for order in orders:
        orders_.append(Order(**order))
    db.session.add_all(orders_)
    db.session.commit()    
    return jsonify({'message': 'Orders added successfully...'}), 201    
```

6. Test the performance and see if it is improved or not. 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(
    'data/retail_db/orders/part-00000',
    names=['order_id', 'order_date', 'order_customer_id', 'order_status']
).drop('order_id')

In [ ]:
orders = df.to_dict(orient='records')

In [ ]:
import requests
import json

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
batch_size = int(input('Enter batch size: '))

In [ ]:
len(orders)

In [ ]:
for i in range(0, len(orders), batch_size):
    orders_chunk = orders[i:i+batch_size]
    response = requests.post(
        f'{base_url}/orders', 
        data={'orders': json.dumps(orders_chunk)}
    )
    print(response.status_code)